<a href="https://colab.research.google.com/github/valsson-group/UNT-Chem3520/blob/main/Python/DFT_MolecularOrbitals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Density Functional Theory Calculations

Simple Juypter notebook that performs Density Functional Theory (DFT) calculations using [pySCF](https://pyscf.org/index.html) running on Google Colab.

A Jupyter notebook is way to run python code where you can run python code in cells. To run through the notebook, you will need to run it cell by cell.




## Setup python environment, imports, and define functions

Here we setup the python environment by installing the needed packages and importing the relevant python packages. We also define a few functions that we will use in our DFT calculations and analysis.

In [ ]:
%%capture
!pip install numpy
!pip install matplotlib
!pip install rdkit
!pip install py3Dmol
!pip install ipywidgets
!pip install fortecubeview
!pip install pythreejs
!pip install pyscf

In [ ]:
import pathlib

# RDKit imports:
from rdkit import Chem
from rdkit.Chem import AllChem, rdCoordGen
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.ipython_useSVG = True  # Use higher quality images for molecules

# For visualization of molecules and orbitals:
import py3Dmol
import fortecubeview

# pyscf imports:
from pyscf import gto, scf, dft, tddft, tools

from pyscf.data.nist import HARTREE2EV
from pyscf.data.nist import HARTREE2WAVENUMBER

# For plotting
import matplotlib
from matplotlib import pyplot as plt

# For numerics:
import numpy as np

from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
def get_xyz_coord(molecule, optimize=False):
    """Get xyz-coordinates for the molecule"""
    mol = Chem.Mol(molecule)
    mol = AllChem.AddHs(mol, addCoords=True)
    AllChem.EmbedMolecule(mol)
    if optimize:  # Optimize the molecules with the MM force field:
        AllChem.MMFFOptimizeMolecule(mol)
    xyz = []
    for lines in Chem.MolToXYZBlock(mol).split("\n")[2:]:
        strip = lines.strip()
        if strip:
            xyz.append(strip)
    xyz = "\n".join(xyz)
    return mol, xyz

def run_dft_calculations(xyz, functional="b3lyp", basis="sto-3g"):
    """Calculate the energy (+ additional things like MO coefficients) with pyscf."""
    mol = gto.M(
        atom=xyz,
        basis=basis,
        unit="ANG",
        symmetry=True,
    )
    mol.build()
    mf = dft.RKS(mol)
    mf.xc = functional
    mf.kernel()
    return mf, mol

def print_homo_lumo_energies(max_homo_lumo=5):

    # find index of HOMO and LUMO
    lumo = float("inf")
    lumo_idx = None
    homo = -float("inf")
    homo_idx = None
    for i, (energy, occ) in enumerate(zip(mf.mo_energy, mf.mo_occ)):
        if occ > 0 and energy > homo:
            homo = energy
            homo_idx = i
        if occ == 0 and energy < lumo:
            lumo = energy
            lumo_idx = i

    # print(f"HOMO (index): {homo_idx}")
    # print(f"LUMO (index): {lumo_idx}")
    # print("")
    print("Molecular Orbitals Energy")
    for i in reversed(range(1,max_homo_lumo)):
      print("- HOMO-{:1}  (MO #{:2d}):  {:7.4f} Hartree".format(i,homo_idx+1-i,  mf.mo_energy[homo_idx-i]))
    print("- HOMO    (MO #{:2d}):  {:7.4f} Hartree".format(homo_idx+1,  mf.mo_energy[homo_idx]))
    print("--------------------------------------")
    print("- LUMO    (MO #{:2d}):  {:7.4f} Hartree".format(homo_idx+1+1,  mf.mo_energy[homo_idx+1]))
    for i in range(1,max_homo_lumo):
      print("- LUMO+{:1}  (MO #{:2d}):  {:7.4f} Hartree".format(i,homo_idx+2+i,  mf.mo_energy[homo_idx+1+i]))

def write_cube_files(
    max_homo_lumo=5, prefix="", dirname=".", margin=5, write_all_orbitals=False
):
    """Write cube files for the given coefficients."""
    path = pathlib.Path(dirname)
    path.mkdir(parents=True, exist_ok=True)

    # find index of HOMO and LUMO
    lumo = float("inf")
    lumo_idx = None
    homo = -float("inf")
    homo_idx = None
    for i, (energy, occ) in enumerate(zip(mf.mo_energy, mf.mo_occ)):
        if occ > 0 and energy > homo:
            homo = energy
            homo_idx = i
        if occ == 0 and energy < lumo:
            lumo = energy
            lumo_idx = i

    #print(f"HOMO (index): {homo_idx}")
    #print(f"LUMO (index): {lumo_idx}")
    #print("")


    if(write_all_orbitals):
      for i in range(mf.mo_coeff.shape[1]):
        outfile = f"{prefix}Orbital-{i:02d}.cube"
        outfile = path / outfile
        print(f"Writing {outfile}")
        tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, i], margin=margin)
    else:
      print("")
      outfile = f"{prefix}HOMO.cube"
      outfile = path / outfile
      print(f"Writing {outfile}")
      tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, homo_idx], margin=margin)

      outfile = f"{prefix}LUMO.cube"
      outfile = path / outfile
      print(f"Writing {outfile}")
      tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, lumo_idx], margin=margin)

      for i in range(1,max_homo_lumo+1):
        outfile = f"{prefix}HOMO_minus-{i:02d}.cube"
        outfile = path / outfile
        print(f"Writing {outfile}")
        tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, homo_idx-i], margin=margin)

        outfile = f"{prefix}LUMO_plus-{i:02d}.cube"
        outfile = path / outfile
        print(f"Writing {outfile}")
        tools.cubegen.orbital(mol, outfile, mf.mo_coeff[:, lumo_idx+i], margin=margin)


## Definition of Molecule from SMILES String

**Here you should define the smiles string for the molecule you want to consider**




In [ ]:
molecule_smiles = "CC(=O)C=C" #@param {type:"string"}

molecule_name = "Mol"
molecule = Chem.MolFromSmiles(molecule_smiles)  # Generate the molecule from smiles
molecule

## Setup of Molecule

In [ ]:
molecule3d, xyz = get_xyz_coord(molecule, optimize=True)
print("XYZ Coordinates obtained from SMILES string {:s}".format(molecule_smiles))
print("")
print("---------------------------------------")
print(xyz)
print("---------------------------------------")

In [ ]:
view = py3Dmol.view(
    data=Chem.MolToMolBlock(molecule3d),
    style={"stick": {}, "sphere": {"scale": 0.3}},
    width=600,
    height=600,
)
view.zoomTo()

## DFT Calculations

Here we perform the DFT calculations. You will need to select the DFT functional and the basis set to be used from the calculation from the drop down lists.

In [ ]:
DFT_Functional = "B3LYP" #@param {type:"string"} ["B3LYP","PBE0", "CAM-B3LYP", "wB97XD"]
BasisSet = "cc-pVTZ" #@param {type:"string"} ["sto-3g", "cc-pVDZ","cc-pVTZ"]

print("Running DFT calculations")
print("- Functional: {:s}".format(DFT_Functional))
print("- Basis set: {:s}".format(BasisSet))

mf, mol = run_dft_calculations(xyz, functional=DFT_Functional, basis=BasisSet)


## Analysis of Results

In [ ]:
mf.analyze(verbose=4)

In [ ]:
print_homo_lumo_energies()

## Calculate and Visualize Orbitals

Here you calculate and visualize the orbitals.

In the cell for visualizng the orbitals, there might be some error message, but you can ignore them.

In [ ]:
!rm -rf cube_files
write_cube_files(
   dirname="cube_files",
   write_all_orbitals=False
)

In [ ]:
fortecubeview.plot(path="./cube_files/", width=600, height=300, colorscheme='national')